In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3 as lite
engine = lite.connect('db.sqlite3')

In [3]:
def check_contain_chinese(check_str):
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fa5':
            return True
    return False

In [20]:
def crawl_category_list():
    HOST = 'https://tw.buy.yahoo.com'
    res = requests.get(HOST+'/help/helper.asp?p=sitemap')
    res.encoding = 'big5'
    soup = BeautifulSoup(res.text,'lxml')
    a_tag = soup.select('#cl-sitemap a')
    category = []
    href = []
    for a in a_tag:
        category.append(a.text)
        href.append(HOST+a['href'])
    df_category = pd.DataFrame(columns=['category','url'])
    df_category['category'] = category
    df_category['url'] = href
    df_category = df_category[df_category['url'].str.contains('sub')]
    return df_category

In [21]:
df_category = crawl_category_list()
df_category.to_sql('yahoo_category', con=engine, if_exists='append', index=False)
cur = engine.cursor()
cur.execute('SELECT * FROM yahoo_category;').fetchall()

In [ ]:
def crawl_product_list(category_href):
    res = requests.get(category_href)
    soup = BeautifulSoup(res.text,'lxml')
    a_tag = soup.select("a")
    href = []
    for a in a_tag:
        try:
            test = a['href']
            href.append(a['href'])
        except KeyError as e:
            pass
    df_products = pd.DataFrame(columns=['product_title','href'])
    df_products['href'] = href
    
    #取出為商品頁的連結
    chinese = []
    for i in range(0,len(df_products)):
        chinese.append(check_contain_chinese(df_products.loc[i]['href']))
    product_list = df_products[chinese].href.unique()
    return product_list

In [ ]:
def crawl_product(product_href,category,position):
    try:
        res = requests.get(product_href)
        soup = BeautifulSoup(res.text,'lxml')
        price = soup.select('.price')
        real_price = -1
        try:
            title = soup.select('h1')[0].text
        except:
            title = '產品名稱無法抓取'
        for p in price:
            try:
                real_price = int(p['content'])
            except KeyError as e:
                pass
        product = [category,title,real_price,position]
        return product
    except:
        pass

In [ ]:
def crawl_all_yahoo_product():
    counter = 0
    category_list = crawl_category_list()
    for index,category in category_list.iterrows():
        df_category_products = pd.DataFrame(columns=['category','title','price','position'])
        category_title = category['category']
        category_href = category['href'] 
        product_list = crawl_product_list(category_href)
        print('-----------------------')
        print(category_title)
        for idx, product_href in enumerate(product_list):
            try:
                df_category_products.loc[counter] = crawl_product(product_href,category_title,idx)
                counter = counter+1
            except:
                pass
        try:
            df_category_products.to_sql('products', con=engine, if_exists='append')
            print(counter)
            print('此類別資料已存入')
            print('-----------------------')
        except:
            print('something fucking wrong!')

In [ ]:
crawl_all_yahoo_product()

In [ ]:
engine.execute("SELECT * FROM products").fetchall()

In [ ]:
def get_all_category_product(category_title, category_href):
    df_category_products = pd.DataFrame(columns=['category','title','price','position'])
    product_list = crawl_product_list(category_href)
    for idx, product_href in enumerate(product_list):
        df_category_products.loc[idx] = crawl_product(product_href,category_title,idx)
    return df_category_products

In [ ]:
category_list = crawl_category_list()
df_category_products = get_all_category_product(category_list.loc[5]['category'],category_list.loc[5]['href'])
df_category_products.to_sql('yahoo_product', con=engine)
engine.execute("SELECT * FROM products").fetchall()